In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [2]:
columName=['userID','age','gender','occupation','code']

In [3]:
df_users=pd.read_csv('C:/Users/sansi/Desktop/dat/u.user',encoding='Latin',sep='|',names=columName)
df_movies=pd.read_csv('C:/Users/sansi/Desktop/dat/movies.dat',encoding='Latin',sep='::',names=['movie_id', 'title', 'movie_genre'])
df_ratings=pd.read_csv('C:/Users/sansi/Desktop/dat/ratings.dat',encoding='Latin',sep='::',names=['userID','movie_id','rate','tims'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
from sklearn.model_selection import train_test_split


In [5]:
movie_trainset,movie_testset =train_test_split(df_movies, test_size=0.05, random_state=0)
user_trainset,user_testset =train_test_split(df_users, test_size=0.2, random_state=0)
rate_trainset,rate_testset =train_test_split(df_ratings, test_size=0.2, random_state=0)

In [6]:
movie_trainset

,movie_id,title,movie_genre
1547,1588,George of the Jungle (1997),Children's|Comedy
2363,2432,Stepmom (1998),Drama
2355,2424,You've Got Mail (1998),Comedy|Romance
398,402,Open Season (1996),Comedy
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
...,...,...,...
835,846,Flirt (1995),Drama
3264,3333,"Killing of Sister George, The (1968)",Drama
1653,1701,Deconstructing Harry (1997),Comedy|Drama
2607,2676,Instinct (1999),Drama|Thriller


In [7]:
df_movies=df_movies.dropna(how='any')
df_users=df_users.dropna(how='any')
df_ratings=df_ratings.dropna(how='any')
df_rates=df_ratings.copy()
df_movies_test=df_movies.copy()
df_movies_gender=df_movies.copy()
df_movies_populaire=df_movies.copy()
movie_trainset_test=movie_trainset.copy()
movie_trainset=movie_trainset.dropna(how='any')

In [8]:
df_users['age'] = pd.cut(df_users['age'], bins=[18,30, 45,55,65, np.inf], labels=[1,2,3,4,5], right=False)#categorie age

In [9]:
df_ratings['rate'].replace([1, 2,3,4,5],[-1, -1, 1, 1, 1],inplace=True)

In [10]:
df=df_movies.merge(df_ratings,on='movie_id')
df=df_users.merge(df,on='userID')
df.set_index('movie_id')
df_trainset,df_testset =train_test_split(df, test_size=0.1, random_state=0)

In [11]:
df_trainset[df_trainset['age']==1]

,userID,age,gender,occupation,code,movie_id,title,movie_genre,rate,tims
74865,509,1,M,administrator,10011,1721,Titanic (1997),Drama|Romance,1,976295996
119232,770,1,M,student,14216,1461,Vegas Vacation (1997),Comedy,-1,976182730
30177,203,1,F,student,32301,31,Dangerous Minds (1995),Drama,-1,976929358
7434,53,1,M,programmer,55414,269,My Crazy Life (Mi vida loca) (1993),Drama,1,977986238
143684,927,1,M,programmer,55428,1821,"Object of My Affection, The (1998)",Comedy|Romance,1,979434142
...,...,...,...,...,...,...,...,...,...,...
7877,53,1,M,programmer,55414,2459,"Texas Chainsaw Massacre, The (1974)",Horror,1,977980829
70608,476,1,M,student,60440,316,Stargate (1994),Action|Adventure|Sci-Fi,1,976219467
137993,889,1,M,technician,78704,1193,One Flew Over the Cuckoo's Nest (1975),Drama,-1,975247862
108631,712,1,F,student,54901,593,"Silence of the Lambs, The (1991)",Drama|Thriller,1,975601569


In [12]:
df_movies=pd.DataFrame(df_movies.loc[:,['movie_id','movie_genre']])
df_movies = pd.concat([df_movies, df_movies.movie_genre.str.get_dummies(sep='|')], axis=1)
df_movies = df_movies.drop(['movie_genre'],axis=1)
df_movies =pd.DataFrame(df_movies.set_index('movie_id'))

In [13]:
n=df_movies.sum(axis=1) 
df_movies_cat=df_movies.divide(n.iloc[:],axis=0)
#len(nonLIST) pour obtenire la taille de la liste 
df_movies_cat

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.333333,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#ponderation tfidf
n=df_movies.sum(axis=0) 
df_df=pd.Series(n,index=df_movies.columns)
def idf(i):
    k=np.log(df_movies.size/i)
    return k  
df_idf=pd.Series(idf(df_df[0:18]))
df_movies_penderation=df_idf[:]*df_movies
df_movies_penderation.head(10)

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,6.500775,5.629282,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.000000,5.509288,0.000000,5.629282,0.000000,0.000000,0.0,0.000000,6.935227,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.999877,0.0,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,3.775103,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
6,4.934145,0.000000,0.000000,0.000000,0.000000,5.802877,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,4.956256,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.999877,0.0,0.000000,0.0,0.0
8,0.000000,5.509288,0.000000,5.629282,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
9,4.934145,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [15]:
df_ratings=df_ratings.drop('tims',axis=1)
df_users=df_users.drop('code',axis=1)
#build the user profile
#inferer le centre d'interet d'un utilisateur 

In [16]:
users=df_ratings['userID']==5
df_ratings_user=df_ratings[users]
df_poids=df_movies_cat[df_movies_cat.index.isin(df_ratings_user.movie_id)]

In [17]:
#df_poids.values[0][0]*df['rate'].values[0]


In [18]:

df_user_rate=df_poids.T.mul(df['rate']).sum(axis=1)
df_user_rate

Action          6.333333
Adventure       1.333333
Animation       1.666667
Children's      2.250000
Comedy         27.583333
Crime           8.333333
Documentary     4.000000
Drama          56.083333
Fantasy         0.000000
Film-Noir       1.083333
Horror          5.833333
Musical         1.083333
Mystery         2.000000
Romance        10.750000
Sci-Fi          5.583333
Thriller       10.916667
War             0.166667
Western         1.000000
dtype: float64

In [19]:
user_preference=df_user_rate.T

In [20]:
user_preference=pd.DataFrame(user_preference)


In [21]:
c=cosine_similarity(user_preference.T,df_movies_penderation)
R = np.argsort(c)[:, ::-1]#trier les valeur de 


In [22]:
def user_table(i) :
    df=df_trainset[df_trainset['userID']==i]
    
    return df
user_table(2)  

,userID,age,gender,occupation,code,movie_id,title,movie_genre,rate,tims
117,2,3,F,other,94043,1687,"Jackal, The (1997)",Action|Thriller,1,978300174
68,2,3,F,other,94043,457,"Fugitive, The (1993)",Action|Thriller,1,978299773
170,2,3,F,other,94043,3334,Key Largo (1948),Crime|Drama|Film-Noir|Thriller,1,978298958
144,2,3,F,other,94043,2359,Waking Ned Devine (1998),Comedy,1,978299666
121,2,3,F,other,94043,1801,"Man in the Iron Mask, The (1998)",Action|Drama|Romance,1,978300002
...,...,...,...,...,...,...,...,...,...,...
120,2,3,F,other,94043,1792,U.S. Marshalls (1998),Action|Thriller,1,978299941
75,2,3,F,other,94043,593,"Silence of the Lambs, The (1991)",Drama|Thriller,1,978298517
163,2,3,F,other,94043,3105,Awakenings (1990),Drama,1,978298673
106,2,3,F,other,94043,1370,Die Hard 2 (1990),Action|Thriller,1,978299889


In [23]:
df_trainset

,userID,age,gender,occupation,code,movie_id,title,movie_genre,rate,tims
85591,558,4,F,writer,63132,1635,"Ice Storm, The (1997)",Drama,1,976054469
74865,509,1,M,administrator,10011,1721,Titanic (1997),Drama|Romance,1,976295996
135498,877,2,M,other,77504,852,Tin Cup (1996),Comedy|Romance,1,975270851
40749,273,3,F,other,10016,2413,Clue (1985),Comedy|Mystery,1,976777018
66886,450,2,F,educator,11758,1136,Monty Python and the Holy Grail (1974),Comedy,1,976234924
...,...,...,...,...,...,...,...,...,...,...
41993,284,2,M,executive,92629,3564,"Flintstones in Viva Rock Vegas, The (2000)",Children's|Comedy,-1,976576736
97639,655,3,F,healthcare,60657,35,Carrington (1995),Drama|Romance,-1,975699752
95939,644,3,M,retired,63645,1912,Out of Sight (1998),Action|Crime|Romance,1,975720463
117952,761,NaN,M,student,97302,2582,Twin Dragons (Shuang long hui) (1992),Action|Comedy,1,975450755


In [24]:
df_homme=df_trainset[df_trainset['gender']=='M']
df_homme_rate=df_movies_cat.T.mul(df_homme['rate']).sum(axis=1)
df_homme_rate=pd.DataFrame(df_homme_rate)
homme=cosine_similarity(df_homme_rate.T,df_movies_penderation)
h=np.argsort(homme)[:, ::-1]
recommendations_homme = [i for i in h[0] if i not in df_homme['movie_id'].values]
preference_homme=df_movies_gender['title'][recommendations_homme]
preference_homme

1448                               Private Parts (1997)
3382                Guess Who's Coming to Dinner (1967)
2934                Train of Life (Train De Vie) (1998)
2845                                       Molly (1999)
1452                                    Van, The (1996)
                             ...                       
995     Davy Crockett, King of the Wild Frontier (1955)
3242                       Man from Laramie, The (1955)
1138                                       T-Men (1947)
2658                               Killer's Kiss (1955)
1137                                    Raw Deal (1948)
Name: title, Length: 669, dtype: object

In [25]:
df_femme=df_trainset[df_trainset['gender']=='F']
df_femme_rate=df_movies_cat.T.mul(df_femme['rate']).sum(axis=1)
df_femme_rate=pd.DataFrame(df_femme_rate)
femme=cosine_similarity(df_femme_rate.T,df_movies_penderation)
f=np.argsort(femme)[:, ::-1]
recommendations_femme = [i for i in f[0] if i not in df_femme['movie_id'].values]
preference_femme=df_movies_gender['title'][recommendations_femme]
preference_femme

3382                  Guess Who's Coming to Dinner (1967)
1579                            Kicked in the Head (1997)
3493                                     Committed (2000)
1899                           Breakfast Club, The (1985)
341     Adventures of Priscilla, Queen of the Desert, ...
                              ...                        
3541                                    Roustabout (1964)
1138                                         T-Men (1947)
1137                                      Raw Deal (1948)
3223                                Big Combo, The (1955)
2658                                 Killer's Kiss (1955)
Name: title, Length: 990, dtype: object

In [26]:
def age(i):
    global df_age,df_age_rate,age,recommendations_age,preference_age
    if i<=5:
        df_age=df_trainset[df_trainset['age']==i]
        df_age_rate=df_movies_cat.T.mul(df_age['rate']).sum(axis=1)
        df_age_rate=pd.DataFrame(df_age_rate)
        age=cosine_similarity(df_age_rate.T,df_movies_penderation)
        g=np.argsort(age)[:, ::-1]
        recommendations_age = [i for i in g[0] if i not in df_age['movie_id'].values]
        preference_age=df_movies_test['title'][recommendations_age]
    return preference_age

In [27]:
def recommander(userID):
    df_ratings_user=df_trainset[df_trainset['userID']==userID]
    df_poids=df_movies_cat[df_movies_cat.index.isin(df_ratings_user.movie_id)]
    df_user_rate=df_poids.T.mul(user_table(userID).rate).sum(axis=1)
    user_preference=df_user_rate.T
    user_preference=pd.DataFrame(user_preference)
    c=cosine_similarity(user_preference.T,df_movies_penderation)
    R = np.argsort(c)[:, ::-1]
    interet = [i for i in R[0] if i not in user_table(userID)['movie_id'].values]
    centre_interet=df_movies_test['title'][interet]
    df_users_id=df_users[df_users['userID']==userID]
    j=df_users_id['age'].values
    if df_users_id['gender'].values=='M':
        homme_age=age(j[0])[age(j[0]).isin(preference_homme)]
        recommandation=centre_interet[centre_interet.isin(homme_age)]
    else:
        femme_age=age(j[0])[age(j[0]).isin(preference_femme)]
        recommandation=centre_interet[centre_interet.isin(femme_age)]
     
    return recommandation 
        

In [28]:
recommander(2)

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
k=[]    
for l in df_users['userID'].head().values:
    testset=df_testset[df_testset['userID']==i]
    pertinant=testset['title'][testset['title'].isin(recommander(i))]
    testset=testset[testset['rate']==1]
    rappel=pertinant.size/len(testset)
    #rappel=nombre du doc pertinant retourner(pertinant) /nombre totale de doc pertinant
    #precision=nombre de doc pertinant delectionner/nombre totale de doc selectionner
    k.append(rappel)
sum(k)/20

In [ ]:
def precision():
    p=[]    
    for l in df_trainset['userID'].head(100).values:
        testset=df_testset[df_testset['userID']==l]
        tp=testset['title'][testset['title'].isin(recommander(l))]
        testset=testset[testset['rate']==1]
        fp=100
        precision=tp.size/100
    #rappel=nombre du doc pertinant retourner(pertinant) /nombre totale de doc pertinant
    #precision=nombre de doc pertinant delectionner/nombre totale de doc selectionner
        p.append(precision)
    return p  

In [ ]:
precision()

In [ ]:
k=[]
for l in df_users['userID'].tail(100):
    testset=df_testset[df_testset['userID']==l]
    tp=testset['title'][testset['title'].isin(recommander(l))]
    testset=testset[testset['rate']==1]
    t=tp.size
    test=len(testset)
    fn=test-t
    rappel=t/(t+fn)
    #rappel=nombre du doc pertinant retourner(pertinant) /nombre totale de doc pertinant
    #precision=nombre de doc pertinant delectionner/nombre totale de doc selectionner
    k.append(rappel)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_users['userID'].tail(100),k)
plt.plot(df_users['userID'].tail(100),p)
plt.xlabel('users')
plt.ylabel('precision')
plt.title('figure1:precision')
plt.show()

In [ ]:
testset=df_testset[df_testset['userID']==1]
pertinant=testset['title'][testset['title'].isin(recommander(1))]

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(recommander(1).head(11),df_testset['title']))

In [ ]:
def evaluat()
for i in user_trainset.userID.:
    df=df_movies_test.merge(df_ratings[df_ratings['userID']==i],on='movie_id')
    df=user_trainset.merge(df[df['userID']==i],on='userID')
    df.set_index('movie_id')
    df_trainset,df_testset =train_test_split(df, test_size=0.2, random_state=0)
    testset=df_testset[df_testset['rate']==1]
    pertinant=recommander(i)[recommander(i).isin(testset['title'])]
    pertinant
    

In [ ]:
testset

In [ ]:
user_testset['userID'].tolist()

In [ ]:
#la creation de la fonction qui calcule le RMSE:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth): #Root Mean Squared Error
    prediction = prediction[ground_truth.nonzero()].flatten() 
    #.flatten() fusionne les elts des array en un array
    #on attribue a prediction, les résultats des prédictions où on connait le vrais rating cad:
    #prediction: tous nos prédictions sur test; ground_truth.nonzero():les vrais résultats qu'on a dans test
    #on va mettre dans prediction les valeurs qu'on a prédit pour les elts qu'on adéja.
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()#pareil dans ground truth
    return sqrt(mean_squared_error(prediction, ground_truth))
rmse(recommander(10),df_testset)

In [ ]:
def metric(y_test,y_pred):
    from sklearn import metrics
    precision=metrics.precision_score(y_test,y_pred)
    rappel=metrics.recall_score(y_test,y_pred)
    list_metrics=[precision,rappel]
    return list_metrics

In [ ]:
metric(recommander(1).head(20),df_testset.title)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarityfrom tqdm import tqdm_notebook as tqdmclass Evaluator():def __init__(self,k=10,training_ratings=None,testing_ratings=None,book_sim=None,novelty_scores=None):self.k = kself.book_sim = book_simself.novelty_scores = novelty_scoresif training_ratings is not None:self.training_ratings = training_ratingsself.num_users = len(self.training_ratings.user_id.unique())self.num_books = len(self.training_ratings.book_id.unique())if testing_ratings is not None:self.testing_ratings = testing_ratingsself.testing_idx = {}for user_id in tqdm(testing_ratings.user_id.unique()):self.testing_idx[user_id] = \testing_ratings[testing_ratings.user_id == user_id]\.book_id.valuesself.result = {}def _average_precision(self, pred, truth):in_arr = np.in1d(pred, truth)score = 0.0num_hits = 0.0for idx, correct in enumerate(in_arr):if correct:num_hits += 1score += num_hits / (idx + 1)return score / min(len(truth), self.k)def _novelty_score(self, pred):# Recommend the top 10 books in novelty score results in ~11# Crop the score to 10.0 for consistencyreturn min(self.novelty_scores.loc[pred].novelty_score.mean(), 10.0)def _diversity_score(self, pred):matrix = self.book_sim.loc[pred, pred].valuesils = matrix[np.triu_indices(len(pred), k=1)].mean()return (1 -ils) * 10def _personalization_score(self, preds, user_ids, book_ids):if len(user_ids) > 3000:np.random.seed(42)user_ids = np.random.permutation(user_ids)[:3000]df = pd.DataFrame(data=np.zeros([len(user_ids), len(book_ids)]),index=user_ids,columns=book_ids
Appendix 12(2))for user_id in user_ids:df.loc[user_id, preds[user_id]] = 1matrix = sp.csr_matrix(df.values)# calculate similarity for every user's recommendation listsimilarity = cosine_similarity(X=matrix, dense_output=False)# get indicies for upper right triangle w/o diagonalupper_right = np.triu_indices(similarity.shape[0], k=1)# calculate average similaritypersonalization = np.mean(similarity[upper_right])return (1 -personalization) * 10def evaluate(self, model):print("Calculating recommendations:")if len(model.preds) == 0:model.fit(self.training_ratings)preds = model.all_recommendation()user_ids = list(preds.keys())book_ids = np.unique(np.concatenate(list(preds.values())))ap_sum = 0nov_score_sum = 0div_score_sum = 0print("Calculating metrics:")for user_id in tqdm(preds.keys()):pred = preds[user_id]truth = self.testing_idx[user_id]ap_sum += self._average_precision(pred, truth)nov_score_sum += self._novelty_score(pred)div_score_sum += self._diversity_score(pred)self.result[model.name] = {}self.result[model.name]['Mean Average Precision'] = "%.2f%%" % (ap_sum / self.num_users * 100)self.result[model.name]['Coverage'] = "%.2f%%" % (len(book_ids) / self.num_books * 100)self.result[model.name]['Novelty Score'] = "%.2f" % (nov_score_sum / self.num_users)self.result[model.name]['Diversity Score'] = "%.2f" % (div_score_sum / self.num_users)self.result[model.name]['Personalization Score'] = "%.2f" % \self._personalization_score(preds, user_ids, book_ids)def print_result(self):print(pd.DataFrame(self.result).loc[['Mean Average Precision','Coverage','Novelty Score','Diversity Score','Personalization Score']])